In [90]:
""" Midterm 
1. Must in a WORD format to answer the asked questions
2. Must explain your findings. Screenshots alone without clear explanations will not receive points. 
3. Complete the tasks, e.g., import libraries and files in the given document. You may need to install additional libraries to run the program.
4. Any caught plagiarism will lead to 0 points and an F for the final grade. 
5. If I doubt about your answers, I will ask you to run your program in front of me and ask you questions during the class or my office hours. 
If you cannot answer my questions, you will receive a 0 for your midterm.
"""

import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

# Q1 import NLTK
import nltk as nltk
# Q1 import xgboost as xgb
import xgboost as xgb
# Q1 import Word2Vec from gensim.models
import gensim

import re
import numpy as np


from gensim.models import Word2Vec
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize
from sklearn.metrics import accuracy_score

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

from keras.models import Model, Sequential
from keras.layers import Input, Dense, Concatenate, Reshape, Dropout, BatchNormalization, Embedding, Flatten

In [91]:
MAX_SEQUENCE_LENGTH = 100
EMBEDDING_DIM = 16

In [92]:
#Q1 laod the movie_reviews.csv with encoding 'latin-1'
data = pd.read_csv("C:\\Users\\Alexi\\OneDrive - Cleveland State University\\Senior Year 2021-2022\\Data Mining + Analytics\\Midterm/movie_reviews.csv", encoding = 'latin-1')

In [93]:
data.shape

(25000, 2)

In [94]:
data.SentimentText = data.SentimentText.str.lower()

In [95]:
def clean_str(string):
    
    string = re.sub(r"https?\://\S+", '', string)
    string = re.sub(r'\<a href', ' ', string)
    string = re.sub(r'&amp;', '', string) 
    string = re.sub(r'<br />', ' ', string)
    string = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', string)
    string = re.sub('\d','', string)
    string = re.sub(r"can\'t", "cannot", string)
    string = re.sub(r"it\'s", "it is", string)
    return string

In [96]:
data.SentimentText = data.SentimentText.apply(lambda x: clean_str(str(x)))

In [97]:
pd.Series(' '.join(data['SentimentText']).split()).value_counts().head(10)

movie    43558
film     39095
it       30659
one      26509
is       20355
like     20270
good     15099
the      13913
time     12682
even     12656
dtype: int64

In [98]:
stop_words = stopwords.words('english') + ['movie', 'film', 'time']
stop_words = set(stop_words)
remove_stop_words = lambda r: [[word for word in word_tokenize(sente) if word not in stop_words] for sente in sent_tokenize(r)]
data['SentimentText'] = data['SentimentText'].apply(remove_stop_words)

In [99]:
model = Word2Vec(
        data['SentimentText'].apply(lambda x: x[0]),
        epochs=10,
        vector_size=16,
        window=5,
        min_count=5,
        workers=10)

In [100]:
model.wv.most_similar('fun')

[('lighten', 0.8160282373428345),
 ('entertaining', 0.8113067746162415),
 ('enjoyable', 0.8016651272773743),
 ('laughs', 0.7974727153778076),
 ('lighthearted', 0.7962534427642822),
 ('enjoy', 0.7910175919532776),
 ('comedy', 0.7804853320121765),
 ('funny', 0.7662642002105713),
 ('unintentional', 0.7638046741485596),
 ('laugh', 0.7572214603424072)]

In [101]:
pd.crosstab(y_actual, y_pred, margins=True)

Predicted,0,1,All
Actual,,,
0,1799,654,2453
1,613,1934,2547
All,2412,2588,5000


In [102]:
model.wv.save_word2vec_format('movie_embedding.txt', binary=False)

In [103]:
def combine_text(text):    
    try:
        return ' '.join(text[0])
    except:
        return np.nan

In [104]:
data.SentimentText = data.SentimentText.apply(lambda x: combine_text(x))

In [105]:
data = data.dropna(how='any')

In [106]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(list(data['SentimentText']))
sequences = tokenizer.texts_to_sequences(data['SentimentText'])
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 77297 unique tokens.


In [107]:
reviews = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [108]:
def load_embedding(filename, word_index , num_words, embedding_dim):
    embeddings_index = {}
    file = open(filename, encoding="utf-8")
    for line in file:
        values = line.split()
        word = values[0]
        coef = np.asarray(values[1:])
        embeddings_index[word] = coef
    file.close()
    
    embedding_matrix = np.zeros((num_words, embedding_dim))
    for word, pos in word_index.items():
        if pos >= num_words:
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[pos] = embedding_vector
    return embedding_matrix

In [109]:
embedding_matrix = load_embedding('movie_embedding.txt', word_index, len(word_index), EMBEDDING_DIM)

In [110]:
X_train, X_test, y_train, y_test = train_test_split(reviews, pd.get_dummies(data.Sentiment), test_size=0.2, random_state=9)

In [111]:
inp = Input((MAX_SEQUENCE_LENGTH,))
embedding_layer = Embedding(len(word_index),
                    EMBEDDING_DIM,
                    weights=[embedding_matrix],
                    input_length=MAX_SEQUENCE_LENGTH,
                    trainable=False)(inp)
model = Flatten()(embedding_layer)
model = BatchNormalization()(model)
model = Dropout(0.10)(model)
model = Dense(units=256, activation='relu')(model)
model = Dense(units=64, activation='relu')(model)
model = Dropout(0.5)(model)
predictions = Dense(units=2, activation='softmax')(model)
model = Model(inputs = inp, outputs = predictions)

model.compile(loss='binary_crossentropy', optimizer='sgd', metrics = ['acc'])

In [112]:
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 100)]             0         
                                                                 
 embedding_2 (Embedding)     (None, 100, 16)           1236752   
                                                                 
 flatten_2 (Flatten)         (None, 1600)              0         
                                                                 
 batch_normalization_2 (Batc  (None, 1600)             6400      
 hNormalization)                                                 
                                                                 
 dropout_4 (Dropout)         (None, 1600)              0         
                                                                 
 dense_6 (Dense)             (None, 256)               409856    
                                                           

In [113]:
model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs=10, batch_size=256)

Epoch 1/10
79/79 [==============================] - 4s 28ms/step - loss: 0.8297 - acc: 0.5148 - val_loss: 0.7105 - val_acc: 0.5622
Epoch 2/10
79/79 [==============================] - 2s 20ms/step - loss: 0.7390 - acc: 0.5626 - val_loss: 0.6749 - val_acc: 0.6042
Epoch 3/10
79/79 [==============================] - 2s 20ms/step - loss: 0.7008 - acc: 0.5933 - val_loss: 0.6538 - val_acc: 0.6422
Epoch 4/10
79/79 [==============================] - 2s 21ms/step - loss: 0.6751 - acc: 0.6201 - val_loss: 0.6358 - val_acc: 0.6614
Epoch 5/10
79/79 [==============================] - 2s 20ms/step - loss: 0.6530 - acc: 0.6408 - val_loss: 0.6186 - val_acc: 0.6826
Epoch 6/10
79/79 [==============================] - 2s 21ms/step - loss: 0.6342 - acc: 0.6567 - val_loss: 0.6011 - val_acc: 0.7006
Epoch 7/10
79/79 [==============================] - 2s 22ms/step - loss: 0.6137 - acc: 0.6813 - val_loss: 0.5831 - val_acc: 0.7176
Epoch 8/10
79/79 [==============================] - 2s 20ms/step - loss: 0.5951 - a

In [114]:
preds = model.predict(X_test)

In [115]:
accuracy_score(np.argmax(preds, 1), np.argmax(y_test.values, 1))

0.7434

In [116]:
y_actual = pd.Series(np.argmax(y_test.values, axis=1), name='Actual')
y_pred = pd.Series(np.argmax(preds, axis=1), name='Predicted')
pd.crosstab(y_actual, y_pred, margins=True)

Predicted,0,1,All
Actual,,,
0,1747,706,2453
1,577,1970,2547
All,2324,2676,5000


In [118]:
review_num  = 333
print("Review: \n"+tokenizer.sequences_to_texts([X_test[review_num]])[0])
sentiment = "Positive" if np.argmax(preds[review_num]) else "Negative"
print("\nPredicted sentiment = "+ sentiment)
sentiment = "Positive" if np.argmax(y_test.values[review_num]) else "Negative"
print("\nActual sentiment = "+ sentiment)

Review: 
video store years ago rented husband enjoyed much bought vhs enjoyed ever since plot well need going point deserves repeated viewings americans especially n't going get jokes first around know n't funny touching sad ray daughter 's wedding cringe inducing karen calls tony brian attempts kiss finally woman black funny words music good soundtrack 've heard years dancing living room world every performance absolutely perfect bill portrayal ray man one many bad stephen perfect tony lovable player carried karen years appealing look makes women want actors equally brilliant ignore read see gem

Predicted sentiment = Positive

Actual sentiment = Positive
